# exp065  
[Notion](https://www.notion.so/exp065-23e8532369184fe7950e2936de90915c?pvs=4)  
3D-CNNによる固形臓器(Liver, Spleen, Kidney)損傷の検出のために、セグメンテーションした臓器のスライス位置のdataframeを作成し保存.  
copy from: exp005  

In [1]:
import os
import random
import sys
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Tuple
from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import torch
import seaborn as sns
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import matplotlib
from matplotlib import animation, rc
rc('animation', html='jshtml')
matplotlib.rcParams['animation.embed_limit'] = 70

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')
    
from src.segmentation.dataset import TestDataset, load_df
from src.image_processing import windowing
from src.visualization import apply_colormap_to_multilabel_images, animate, print_injury
from src.segmentation.model import load_models
from src.segmentation.trainer import evaluate
from src.image_processing import apply_preprocess
from src.segmentation.model import load_models as seg_load_models
from src.segmentation.trainer import inference as seg_inference

# Segmentation Config

In [2]:
class CFG:
    exp_name = 'exp_004'
    # model config
    backbone = 'efficientnet-b3'
    n_ch = 1
    n_class = 4 # 学習時は腎臓の左右を区別しないので、5->4
    # hyper params
    expand_ch_dim = False
    wl = 0
    ww = 400
    init_lr = 1e-3
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (512, 512)
    batch_size = 32
    amp = True
    n_epoch = 20
    iteration_per_epoch = 200
    pretrain = True
    freeze_epochs = 1
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset001/train_images"
    mask_dir = "data/dataset001/segmentations"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 0
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load DataFrame

In [3]:
df_train = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train.csv')
df_train_image_level = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/image_level_labels.csv')
df_train_series_meta = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')
base_dir = "data/rsna-2023-abdominal-trauma-detection"
dataset_dir = "data/dataset002"

# get label correspondences
organ_index_dict_inv = {
    0: 'liver',
    1: 'spleen',
    2: 'kidney',
    3: 'bowel'
}
organ_index_dict = {v: k for k, v in organ_index_dict_inv.items()}

In [4]:
def get_dataframe():
    """データセットのDataFrameを作成する.
    データセットによって内容を書き換える必要あり.
    """
    # df_train_series_metaをベースに、データフレームを構築.
    image_paths = []
    pid_list = []
    sid_list = []
    for i in range(len(df_train_series_meta)):
        sr = df_train_series_meta.iloc[i]
        pid, sid = sr["patient_id"], sr["series_id"]
        pid, sid = int(pid), int(sid)
        dir_ = f"data/dataset001/train_images/{pid}/{sid}"
        path_list = os.listdir(dir_)
        path_list = [[int(path.replace(".npy","")), path] for path in path_list]
        path_list.sort()
        path_list = [path[1] for path in path_list]
        for path in path_list:
            image_paths.append(os.path.join(dir_, path))
            pid_list.append(pid)
            sid_list.append(sid)
    # 画像データのDataFrameを作成
    df = pd.DataFrame({
            'image_path': image_paths,
            'patient_id': pid_list,
            'series_id': sid_list,
            })
    return df

df = get_dataframe()
# 必要
df["mask_path"] = None

# Load models

In [5]:
# モデルの読み込み
models = load_models(CFG, mode="final")

# Inference Code  
- シリーズごとの推論の準備
- セグメンテーションの前処理及び後処理
- 臓器抽出後の分割・保存

In [6]:
def load_series_from_dataset(dir_: str, max_slices: int=200)-> Tuple[np.ndarray, list]:
    """seriesを読み込む.
    Args:
        dir_ (str): seriesのディレクトリ.
    Returns:
        np.ndarray: (Z, H, W)形式の画像の配列.
        list: 画像IDのリスト.
    """
    path_list = os.listdir(dir_)
    path_list = [[int(path.replace(".npy","")), path] for path in path_list]
    path_list.sort()
    image_id_list = [path[0] for path in path_list]
    path_list = [path[1] for path in path_list]


    if len(image_id_list) > max_slices:
        step = len(image_id_list) // max_slices
        image_id_list = image_id_list[::step]
        path_list = path_list[::step]

    arr = []
    for path in path_list:
        arr.append(np.load(os.path.join(dir_, path)))
    return np.array(arr), image_id_list

def morpho_pytorch(masks):
    for c in range(masks.shape[-1]):
        with torch.no_grad():
            arr = torch.tensor(masks[...,c][np.newaxis]).to(CFG.device).to(torch.float32)
            #dialation
            arr = torch.nn.MaxPool3d(3, stride=1, padding=1, dilation=1, return_indices=False, ceil_mode=False)(arr)
            #erosion
            arr = -torch.nn.MaxPool3d(3, stride=1, padding=1, dilation=1, return_indices=False, ceil_mode=False)(-arr)
        arr = arr.squeeze(0).cpu().numpy().astype(np.uint8)
        masks[...,c] = arr
    return masks

# 各臓器に対して、一定閾値以下のボクセルの集合を切り捨てる
area_th = {
    "liver":50,
    "spleen":20,
    "kidney":20,
    "bowel":30,
}

def area_0fill(masks):
    for idx,mask in enumerate(masks):
        for c,th in area_th.items():
            c_idx = organ_index_dict[c]
            
            contours = cv2.findContours(mask[...,c_idx],cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = contours[0] #0番目は元画像,2番目は階層構造。新しいopencvだと1番目のみがコンツーリング情報っぽい
            for con in contours:
                area = cv2.contourArea(con)
                if area <= area_th[c]:
                    fill_mask = mask[...,c_idx].copy()
                    fill_mask = cv2.drawContours(fill_mask, [con], 0, 0, -1)
                    masks[idx,:,:,c_idx] =  fill_mask
    return masks

def apply_postprocess(mask: np.ndarray)-> np.ndarray:
    """セグメンテーション後の臓器マスクの後処理.
    Args:
        mask (numpy.ndarray): (Z, H, W, C)のマスク画像.
    """
    mask = morpho_pytorch(mask)
    # mask = area_0fill(mask)
    return mask

def evaluate_series(CFG: Any, df: pd.DataFrame, models: list, pid: int, sid: int) -> dict:
    """患者ごと(シリーズごと)の評価を行う.
    Args:
        CFG (Any): Config
        df (pd.DataFrame): get_training_dataframeによって作成したdf
        models (list): 学習済みモデルのリスト
        pid (int): 患者ID
        sid (int): シリーズID
    Returns:
        dict: 評価結果
    """
    # 評価用データセットの作成
    df_res = df[(df["patient_id"] == pid) & (df["series_id"] == sid)].reset_index(drop=True)
    if len(df_res) == 0:
        raise ValueError(f"pid:{pid}, sid:{sid} is not found.")
    ds = TestDataset(CFG, df_res, preprocess=apply_preprocess)
    eval_iterator = DataLoader(
        ds,
        shuffle=False,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers
    )
    # 推論
    result = evaluate(CFG, models, eval_iterator)
    return result

def crop_organ(image: np.ndarray, mask: np.ndarray)-> np.ndarray:
    """臓器のみを切り抜き、臓器に外接するボリュームを返す."""
    # 臓器が存在する部分のインデックスを取得
    z_indices, h_indices, w_indices = np.where(mask != 0)

    """# 各軸に沿って最小と最大のインデックスを見つける
    z_min, z_max = np.min(z_indices), np.max(z_indices)
    h_min, h_max = np.min(h_indices), np.max(h_indices)
    w_min, w_max = np.min(w_indices), np.max(w_indices)"""

    # 各軸に沿って、p%のボクセルが含まれる範囲を見つける
    p = 98
    z_min, z_max = np.percentile(z_indices, 100-p), np.percentile(z_indices, p)
    h_min, h_max = np.percentile(h_indices, 100-p), np.percentile(h_indices, p)
    w_min, w_max = np.percentile(w_indices, 100-p), np.percentile(w_indices, p)
    z_min, z_max = int(z_min), int(z_max)
    h_min, h_max = int(h_min), int(h_max)
    w_min, w_max = int(w_min), int(w_max)

    # この範囲でセグメンテーションデータを切り抜く
    margin = 10
    z_min, z_max = max(0, z_min - 5), min(image.shape[0], z_max + 5)
    h_min, h_max = max(0, h_min - margin), min(image.shape[1], h_max + margin)
    w_min, w_max = max(0, w_min - margin), min(image.shape[2], w_max + margin)
    cropped_image = image[z_min:z_max+1, h_min:h_max+1, w_min:w_max+1]
    cropped_mask = mask[z_min:z_max+1, h_min:h_max+1, w_min:w_max+1]

    # crop segmentation
    # cropped_image = cropped_image * cropped_mask + (1 - cropped_mask) * -1000

    return cropped_image, cropped_mask

def kidney_split(image: np.ndarray, mask: np.ndarray)-> np.ndarray:
    """腎臓について、一度crop_organに入力したものを再度この関数に入力することで左右の腎臓に切り出す.
    Note:
        本関数中のleft/rightは画像上のleft_rightを表す.
    """
    w_half = image.shape[2] // 2
    left_image = image[:, :, :w_half]
    left_mask = mask[:, :, :w_half]
    right_image = image[:, :, w_half:]
    right_mask = mask[:, :, w_half:]
    left_image, _ = crop_organ(left_image, left_mask)
    right_image, _ = crop_organ(right_image, right_mask)
    return left_image, right_image

def resize_volume(mask: np.ndarray, hw_shape: tuple)-> np.ndarray:
    """h, wが512ではない場合にmaskをimageに合うようにリサイズする."""
    new_arr = []
    for i in range(mask.shape[0]):
        new_arr.append(cv2.resize(mask[i], hw_shape[::-1]))
    return np.stack(new_arr)

In [7]:
class Inference:
    """推論パイプライン."""
    def __init__(self, CFG_SEG: Any):
        self.CFG_SEG = CFG_SEG

        self.seg_models = seg_load_models(CFG_SEG, mode="final")
    
    def __call__(self, pid: int, sid: int) -> tuple:
        df_study = df_train_series_meta[(df_train_series_meta['patient_id']==pid) & (df_train_series_meta['series_id']==sid)].reset_index(drop=True)
        preds = defaultdict(list)
        for sid in df_study['series_id'].to_list()[:1]:
            image, image_id_list = self.load_data(pid, sid)

            masks = self.get_segmentation(image)

        return image, image_id_list, masks
        


    def load_data(self, pid: int, sid: int)-> np.ndarray:
        """dicomから画像を読み込む.
        Args:
            pid (int): patient id.
            sid (int): series id.
        Returns:
            np.ndarray: (Z, H, W) normalized CT series.
        Note:
            - preprocessは全モデル共通なので、ここで行う.
            - H, Wはすべてself.CFG_INF.image_sizeにresizeされる.
        """
        image_dir = f"data/dataset001/train_images/{pid}/{sid}"
        image, image_id_list = load_series_from_dataset(image_dir, max_slices = 200) # ここで最大スライス数も制限
        # 現状、モデルごとにwindowing処理が異なるため、ここでのpreprocessはresizeのみ.
        image = apply_preprocess(image, resize=self.CFG_SEG.image_size, do_windowing=False)
        return image, image_id_list
    
    def get_segmentation(self, data: np.ndarray)->np.ndarray:
        """Segmentation modelを使って、各臓器のマスクを作成.
        Args:
            data: (Z, H, W).
        Returns:
            mask: (z, h, w, ch) binarized."""
        seg_iterator = self.pseudo_iterator(self.CFG_SEG, data)
        pred = seg_inference(self.CFG_SEG, self.seg_models, seg_iterator)
        pred = (pred > 0.5).astype(np.uint8)
        return pred
        
    def pseudo_iterator(self, CFG: Any, images: np.ndarray)-> tuple:
        """evaluation iterator.
        Args:
            CFG: config.
            images: (batch dim, H, W) or (batch dim, Z, H, W).
        """
        # モデルごとにwindowingが異なるため、ここで行う.
        images = apply_preprocess(images, wl=CFG.wl, ww=CFG.ww)
        batch = CFG.batch_size
        length = len(images)
        arr = []
        if not CFG.expand_ch_dim:
            images = self.add_dummy_array(CFG, images)
        for i in range(length):
            if CFG.expand_ch_dim:
                img = images[i]
                img = img[np.newaxis, ...]
            else:
                img = images[i:i+CFG.n_ch]
            arr.append(img)
            if i != 0 and (i%batch==0 or i == length-1):
                arr = np.stack(arr, axis=0)
                arr = torch.from_numpy(arr.astype(arr.dtype, copy=False))
                yield arr
                arr = []
    def add_dummy_array(self, CFG: Any, images: np.ndarray)-> np.ndarray:
        """chが複数ある場合に、事前に0配列を追加しておく."""
        add_ch = CFG.n_ch//2
        arr = []
        img = np.zeros_like(images[0])
        for i in range(add_ch):
            arr.append(img)
        arr.extend(images)
        for i in range(add_ch):
            arr.append(img)
        arr = np.stack(arr, axis=0)
        return arr

In [8]:
df_train_series_meta

,patient_id,series_id,aortic_hu,incomplete_organ
0,10004,21057,146.00,0
1,10004,51033,454.75,0
2,10005,18667,187.00,0
3,10007,47578,329.00,0
4,10026,29700,327.00,0
...,...,...,...,...
4706,9961,2003,381.00,0
4707,9961,63032,143.75,0
4708,9980,40214,103.00,0
4709,9980,40466,135.00,0


In [9]:
# liver, spleen, kidneyのいずれかに損傷がある患者のみ抽出
injury_pids = df_train[
    (df_train["liver_low"] == 1) | 
    (df_train["liver_high"] == 1) | 
    (df_train["kidney_low"] == 1) |
    (df_train["kidney_high"] == 1) |
    (df_train["spleen_low"] == 1) |
    (df_train["spleen_high"] == 1)
    ]["patient_id"].unique()
# df_train_series_metaから、該当するpatient_idのみ抽出
df_train_series_meta_injury_lsk = df_train_series_meta[df_train_series_meta["patient_id"].isin(injury_pids)].reset_index(drop=True)

In [12]:
def inference() -> None:
    """学習用全データに対するセグメンテーション推論を行う.
    切り抜いた臓器のCT画像を保存する.
    保存は、f'{pid}_{sid}_{organ}.npy'という形式で、スライス位置などの情報を付加せずに外接矩形で保存.
    ボリュームサイズは制限せず、元の解像度で保存.
    """
    result_list = []
    inference_instance = Inference(CFG)
    for i in tqdm(range(len(df_train_series_meta))):# tqdm(range(len(df_train_series_meta_injury_lsk))):
        # pid, sid = df_train_series_meta_injury_lsk.iloc[i][["patient_id", "series_id"]]
        pid, sid = df_train_series_meta.iloc[i][["patient_id", "series_id"]]
        pid, sid = int(pid), int(sid)
        image, image_id_list, masks = inference_instance(pid, sid)
        # z軸がinstance_numberとあっていることを確認
        assert len(image_id_list) == masks.shape[0]
        result = {
            "patient_id": pid,
            "series_id": sid,
        }
        for organ in ["kidney", "liver", "spleen"]:
            organ_idx = organ_index_dict[organ]
            organ_segment = masks[..., organ_idx]
            # z軸のorgan_segmentが少なくとも1以上である最小と最大のインデックスを取得
            # 臓器が存在する部分のインデックスを取得
            # print(organ_segment.shape)
            z_indices, h_indices, w_indices = np.where(organ_segment != 0)
            # print(z_indices)
            # この部分が適用される例：
            # 腎臓が片方しか無く、左右分割したあとに全くボクセルがない方が存在する場合
            if z_indices.shape[0] == 0:
                result[f"{organ}_zmin"] = -1
                result[f"{organ}_zmax"] = -1

            # 各軸に沿って、p%のボクセルが含まれる範囲を見つける
            else:
                p = 99
                z_min, z_max = np.percentile(z_indices, 100 - p), np.percentile(z_indices, p)
                z_min, z_max = int(z_min), int(z_max)
                result[f"{organ}_zmin"] = image_id_list[z_min]
                result[f"{organ}_zmax"] = image_id_list[z_max]

        result_list.append(result)
    return result_list

In [13]:
result_list = inference()

100%|██████████| 4711/4711 [3:26:36<00:00,  2.63s/it]  


In [14]:
df = pd.DataFrame(result_list)

In [15]:
df.head()

,patient_id,series_id,kidney_zmin,kidney_zmax,liver_zmin,liver_zmax,spleen_zmin,spleen_zmax
0,10004,21057,571,756,361,691,396,651
1,10004,51033,572,752,367,692,402,662
2,10005,18667,55,77,37,66,43,54
3,10007,47578,69,84,53,72,56,72
4,10026,29700,109,247,71,239,59,145


In [16]:
os.makedirs("outputs/exp065", exist_ok=True)
df.to_csv("outputs/exp065/segmentation_result.csv", index=False)